# Vehicle Emissions Estimate

In [1]:
import pandas as pd
import sqlite3 as sql
import numpy as np
import re
import string

### Create combined vehicle emissions:

1. join curb_weight_join to weighted total c02 emissions by vehicle
2. need to get embodied emissions and in-use emissions 
3. embodied_emissions = vehicleCo2_vector x vehicle_C02_weightings x mass_vehicle
4. in-use emissions = Co2/mile = mpg(e) x carbon_intensity
5. carbon intensity = carbon intensity of fuel or carbon intensity of grid_mix 
6. grid_mix carbon_intensitty = c02 emissions at entered zip code or take average of all zip codes
7. append in-use emissions, embodied emissions, and total emissions (embodied+in_use) to joined vehicle data

## Load in Data

In [2]:
## load in data
## need vehicle data, emissions weightings/intensities by vehicle, zip data
vehicle_data = pd.read_csv("../Data/adj_weight_data_join_2000-2023.csv")
subregion_emissions = pd.read_csv("../Data/subregion_emissions.csv")

## embodied emissions data (includes both weightings and intensities in the same table)
LI_emissions_data = pd.read_csv("../Data/emissions_factor_LI.csv")
NIMH_emissions_data = pd.read_csv("../Data/emissions_factor_NIMH.csv")
vehicle_emissions_data = pd.read_csv("../Data/emissions_factor_vehicle.csv")

In [3]:
## make sure emissions column names are formatted properly
LI_emissions_data.columns = [i.strip(" ,") for i in LI_emissions_data.columns]
NIMH_emissions_data.columns = [i.strip(" ,") for i in NIMH_emissions_data.columns]
vehicle_emissions_data.columns = [i.strip(" ,") for i in vehicle_emissions_data.columns]

In [10]:
## save csvs with formatted column names
## embodied emissions data (includes both weightings and intensities in the same table)
LI_emissions_data.to_csv("../Data/emissions_factor_LI.csv", index = False)
NIMH_emissions_data.to_csv("../Data/emissions_factor_NIMH.csv", index = False)
vehicle_emissions_data.to_csv("../Data/emissions_factor_vehicle.csv", index = False)

## Subregion Grid Mixes

In [5]:
##Subregion Grid Mixes (CO2eRate)
subregion_emissions[['Region_code','CO2eRate']]

,Region_code,CO2eRate
0,AKGD,1120.813
1,AKMS,551.276
2,AZNM,956.883
3,CAMX,455.251
4,ERCT,872.389
5,FRCC,864.535
6,HIMS,1195.598
7,HIOA,1707.576
8,MROE,1512.599
9,MROW,1106.448


### Average US Grid Mix

In [6]:
## first average subregion emissions (use CO2e)
## in this table, values are in lbs/MWh
avg_CO2eRate = subregion_emissions['CO2eRate'].mean()
## convert from lbs/MWh to g/kWh
avg_CO2eRate = avg_CO2eRate*453.592/1000

## Emissions Estimate Functions

In [13]:
## define functions to import as a file
## Vehicle is planned object which will be one entry taken from EPA vehicle testing database
## User is planned object which will store a user's input

def est_battery_capacity_from_vehicle(Vehicle):
    '''return battery capacity in kwh'''
    kwh_size = 0
    if Vehicle.atvType == 'EV':
        kwh_size = Vehicle.Range*(1/Vehicle.highway08)*33.7 ## use Range since all EVs have it
    elif Vehicle.atvType == 'PHEV':
        kwh_size = Vehicle.rangeHwyA*(Vehicle.combE/100) ## range*(kWh/100 miles) = kWh estimated
#     print(kwh_size)
    return kwh_size

def get_HEV_battery_weight(vehicle_weight, vehicle_year):
    if vehicle_year < 2020: ## for vehicles before 2020, assume NIMH, after assume Li-ion (lighter weight)
        weight_factor = 1
    else:
        weight_factor = 0.485 ## From GREET 2 NIMH/LI-ion battery weight estimates
    ## simple clustering by vehicle weight 
    ## estimated from HEV battery replacement data at https://www.besthybridbatteries.com
    ## adjust by weight factor 2020 model year and after
    if vehicle_weight < 3300: 
        return 75*weight_factor
    elif vehicle_weight > 3300 and vehicle_weight < 4000:
        return 120*weight_factor
    else:
        return 150*weight_factor
    
def est_battery_weight_from_vehicle(Vehicle):
    ''' return battery weight in lbs'''
    if Vehicle.atvType == 'HEV':
        return get_HEV_battery_weight(Vehicle.AdjWeight, Vehicle.Year)
    else:
        return est_battery_capacity_from_vehicle(Vehicle)*1/(.2)*2.204 ## assume 0.2 kWh/Kg energy density

def est_battery_assembly_emissions(Vehicle, emittant):
    battery_assembly_dict = {'VOC': 2.24, 'CO': 9.129,'NOx': 13.081,'PM10': 0.939,
     'PM2.5': 0.747,'SOx': 3.981,'BC': 0.099,'OC': 0.279,'CH4': 37.428,'N2O': 0.361,
     'CO2': 12374}
    return est_battery_capacity_from_vehicle(Vehicle)*battery_assembly_dict[str(emittant)]

def weighted_emissions_calc(vehicle_weight, mass_fractions, emissions_vector):
    mass_fractions = np.array(mass_fractions)
    emissions_vector = np.array(emissions_vector)
    return np.sum(vehicle_weight*mass_fractions*emissions_vector)

H2w = [0.77, 0.165, 0.165] ## from CA standard: 33% 'renewable',assumed as 50/50 green elecrtrolysis/biomethane
## gCo2e/MJ https://ww2.arb.ca.gov/sites/default/files/classic/fuels/lcfs/ca-greet/lut-doc.pdf
H2CO2e = [117.67, 99.48, 10.51] 
def get_FCV_usage_emissions(vehicle_efficiency, miles_estimated, 
                                hydrogen_weights=H2w, emissions_vector_hydrogen=H2CO2e):
    '''assumes vehicle efficiency entered in mpge'''
    ## model as efficiency*distance*emissions_intensity
    ## emissions_intensity is weighted sum of emissions intensities
    total_intensity = np.sum(np.array(hydrogen_weights)*np.array(emissions_vector_hydrogen))
    ## mpg ~ m/kgH2
    ## 120 MJ/kg H2 (energy.gov)
    usage_emissions = (1/vehicle_efficiency)*miles_estimated*total_intensity*120
    return usage_emissions
    
    
def get_HEV_battery_emissions(battery_weight, NIMH_emissions_manufacturing, 
                             LI_emissions_manufacturing, emissions_type, vehicle_type):
    if battery_weight > 75: ## then it is a NIMH battery based on grouping and scaling factor
        return weighted_emissions_calc(battery_weight, NIMH_emissions_manufacturing[str(emissions_type)], 
                                       NIMH_emissions_manufacturing[str(vehicle_type)])
    else:
        return weighted_emissions_calc(battery_weight, 
                                       LI_emissions_manufacturing[str(emissions_type)], 
                                       LI_emissions_manufacturing[str(vehicle_type)])
def get_fuel_carbon():
    ## future option: 
    #     np.sum(total_fuel*(np.array(fuel_import_region_weights)*np.array(upstream_ghg_intensities)))
    ##
    carbon_per_MJ = 90.5432 ## gCO2e/MJ from GREET WTW calculator US average
    energy_per_gallon = 118227 ## BTU/gallon averaged from https://afdc.energy.gov/files/u/publication/fuel_comparison_chart.pdf
    ## now convert from carbon per MJ to carbon per gallon (.001055 btu per MJ)
    return carbon_per_MJ*energy_per_gallon*.001055  ## from GREET WTW calculator, GREET1 model with US fuel mix

## emissions in grams per ton of battery recycled
co2_batt = [2109062, 1909883, 687417, 553818] 
ch4_batt = [2311.495, 4519.279, 1085.902, 705.049]
n2O_batt = [21.374, 38.675, 36.109, 6.781]
battery_recycling_emissions_dict = {"CO2": co2_batt, "CH4": ch4_batt, "N2O": n2O_batt}
def get_battery_recycling_emissions(battery_weight, emittant,
                                    method_weights = [.25, .30, .30, .15], emittant_dict=battery_recycling_emissions_dict):
    '''gets battery recycling emissions, takes battery_weight in lbs and converts to tons.'''
    battery_weight = battery_weight/2000 ## convert to tons from lbs
    
    emissions_sum = np.sum(battery_weight*np.array(method_weights)*np.array(emittant_dict[str(emittant)]))
    return emissions_sum
    
def get_charging_loss(percent_lost = 16.6, grid_loss = 5.5):
    ## assume charging loss of 16.6% from: https://www.mdpi.com/2624-8921/3/4/43
    ## assume grid loss of 5.5% calculate from: https://www.eia.gov/totalenergy/data/flow-graphs/electricity.php
    return (1/(1-(percent_lost*.01)))*(1/(1-(grid_loss*.01)))

def get_PHEV_usage_emissions(Vehicle, miles_estimated, grid_intensity):
    '''implement PHEV emissions calculation using EPA utility factor'''
    utility_factor = Vehicle.combinedUF
    electric_eff = Vehicle.combE ## in kwh/100 miles (average energy consumption in both hybrid and EV modes)
    permile_electric_eff = electric_eff/100
    combined_mpge = Vehicle.comb08
    
    grid_carbon = grid_intensity*permile_electric_eff*miles_estimated*get_charging_loss()
    fuel_carbon = (1/combined_mpge)*miles_estimated*get_fuel_carbon() ## consumption in hybrid mode (EV mode is zero fuel)
    
    
    return 1/((1/grid_carbon) + (1-utility_factor)/fuel_carbon) ## electric_eff always applies (no weight)   
    
##
## update to also work with lightweight versions correctly?
##
def get_embodied_emissions(vehicle_weight, battery_weight, vehicle_type, 
                           emissions_type, vehicle_emissions_manufacturing, 
                           LI_emissions_manufacturing, NIMH_emissions_manufacturing):
    if vehicle_type == 'ICEV: Conventional Material':
        battery_sum = 0
        fluids = 634200
    elif vehicle_type == 'HEV: Conventional Material':
        battery_sum = get_HEV_battery_emissions(battery_weight, 
                                                NIMH_emissions_manufacturing,
                                                LI_emissions_manufacturing, emissions_type, vehicle_type)
        fluids = 634200
    elif vehicle_type == 'PHEV: Conventional Material':
        battery_sum = weighted_emissions_calc(battery_weight, LI_emissions_manufacturing[str(emissions_type)], 
                                       LI_emissions_manufacturing[str(vehicle_type)]) 
        fluids = 571000
    elif vehicle_type == 'FCV: Conventional Material':
        ## assume 1.5 kWh Li-ion battery ~100 lbs (reasonable based on Mirai battery)
        battery_weight = 100
        battery_sum = weighted_emissions_calc(battery_weight, LI_emissions_manufacturing[str(emissions_type)], 
                                              LI_emissions_manufacturing[str(vehicle_type)]) 
        fluids = 167000
    else:
        battery_sum = weighted_emissions_calc(battery_weight, LI_emissions_manufacturing[str(emissions_type)], 
                                              LI_emissions_manufacturing[str(vehicle_type)])
        fluids = 82000
        
    net_vehicle_weight = vehicle_weight - battery_weight
    vehi_sum = weighted_emissions_calc(net_vehicle_weight, vehicle_emissions_manufacturing[str(emissions_type)], 
                                       vehicle_emissions_manufacturing[str(vehicle_type)])
    if emissions_type == 'CO2':
        ## terms assumed constant:
        lead_acid_battery = 60000
        vehicle_adr = 874000
        battery_sum+=lead_acid_battery
        return battery_sum, vehi_sum, (vehi_sum + battery_sum + fluids + vehicle_adr)
    else: ## assume for other emittants the constant terms are negible (don't add them)
        return battery_sum, vehi_sum, (vehi_sum + battery_sum)
        

# def est_embodied_emissions(Vehicle):
#     ## account for EV battery or ICE
#     if Vehicle.battery == True:
#         battery = True
#         kwh_size = est_battery_capacity_from_vehicle(Vehicle)
#     else:
#         battery = False
#         kwh_size = 0
#     vehicle_weight = Vehicle.weight
#     ## send estimated battery capacity and vehicle weight to embodied emissions function
#     embodied_emissions = get_embodied_emissions(vehicle_weight, battery_weight)  
#     return embodied_emissions

# def get_grid_carbon(user_zip, Zip, Co2):
#     ## store lookup table of CO2 intensity by grid region
#     ## convert User zipcode input to grid region
# #     user_region = Zip[Zip['ZIP'] == user_zip]
# #     Co2 = Co2[Co2['Co2'] == user_region]
#     return Co2
    

def get_usage_emissions(Vehicle, vehicle_fuel_economy, vehicle_class, 
                        miles_estimated, grid_intensity):

    if vehicle_class == 'ICE' or vehicle_class == 'HEV':
        return (1/vehicle_fuel_economy)*miles_estimated*get_fuel_carbon() ## (g/mile*miles*gCo2/gallon)
    elif vehicle_class == 'EV':
        grid_carbon = grid_intensity*((vehicle_fuel_economy/33.7)**-1)*miles_estimated*get_charging_loss() ## convert mpge to kwh/mile, return gC
        return grid_carbon ## carbon per kwh*kwh/gallon
    elif vehicle_class == 'PHEV': ## assume vehicle is driven in combined cycle (battery and engine)
        return get_PHEV_usage_emissions(Vehicle, miles_estimated, grid_intensity) 
    elif vehicle_class == 'FCV':
        return get_FCV_usage_emissions(vehicle_fuel_economy, miles_estimated, 
                                       hydrogen_weights=H2w, emissions_vector_hydrogen=H2CO2e)
    else:
        return 0

# def est_lifetime_emissions(Vehicle, User):
#     ## account for EV battery or ICE
#     embodied_emissions = est_embodied_emisions(Vehicle)
#     usage_emissions = est_usage_emissions(Vehicle,User)
#     return embodied_emissions+usage_emissions

def conv_or_lw(vehicle_class, lightweight=False):
    '''input vehicle_class (Vehicle.atvType) and Boolean (lightweight = False for conventional)'''
    
    possible_columns = ['ICEV: Conventional Material', 'ICEV: Lightweight Material',
       'HEV: Conventional Material', 'HEV: Lightweight Material',
       'PHEV: Conventional Material', 'PHEV: Lightweight Material',
       'EV: Conventional Material', 'EV: Lightweight Material',
       'FCV: Conventional Material', 'FCV: Lightweight Material']
    
    if not lightweight:
        if vehicle_class == 'ICE':
            return possible_columns[0]
        elif vehicle_class == 'HEV':
            return possible_columns[2]
        elif vehicle_class == 'PHEV':
            return possible_columns[4]
        elif vehicle_class == 'EV':
            return possible_columns[6]
        else:
            return possible_columns[8]
    elif lightweight:
        if vehicle_class == 'ICE':
            return possible_columns[1]
        elif vehicle_class == 'HEV':
            return possible_columns[3]
        elif vehicle_class == 'PHEV':
            return possible_columns[5]
        elif vehicle_class == 'EV':
            return possible_columns[7]
        else:
            return possible_columns[9]
    else:
        print("there was an error with get_emissions_weights lightweight parameter")
        return
                
emittant_dictionary = {'CO2':1, 'CH4':25, 'N2O':298} ## from IPCC report (wikipedia/GWP)
def get_CO2e(value, emittant, conversions=emittant_dictionary):
    '''given a GHG emittant and a emissions value, return CO2e.
    Use global warming potential at 100 years as a scaling factor'''
#     print(value)
    return value*conversions[emittant]
    
def process_vehicle(Vehicle, Manufacturing, LI_Battery, NIMH_Battery, emissions_types, miles_estimated, grid_intensity):
    vehicle_fuel_economy = Vehicle.highway08*.55 + Vehicle.city08*.45 ## combined fuel economy (EPA.gov)
    vehicle_weight = Vehicle.AdjWeight
    vehicle_class = Vehicle.atvType
    vehicle_year = Vehicle.Year
    vehicle_type = conv_or_lw(vehicle_class)
    battery_weight = est_battery_weight_from_vehicle(Vehicle)
    
    lifetime_emissions = []
    embodied_emissions = {"battery_assembly":0, "battery":0, "vehicle_assembly":0, "total_embodied":0}
    for emittant in emissions_types:
        
        ## get battery, vehicle, and battery and vehicle emissions associated with a given emittant
        battery, vehicle, total = get_embodied_emissions(vehicle_weight, battery_weight, vehicle_type, 
                           emittant, Manufacturing, 
                           LI_Battery, NIMH_Battery)
        ## get battery assembly emissions 
        batt_assembly = est_battery_assembly_emissions(Vehicle, emittant)
        ## get battery_recycling emissions (assumes LI battery)
        batt_recycling = get_battery_recycling_emissions(battery_weight, emittant)
        
        ## add emissions for each emittant to totals
        ## convert each emittant to CO2e
        embodied_emissions["battery_assembly"]+=get_CO2e(batt_assembly, emittant)
        embodied_emissions["battery"]+=get_CO2e(battery, emittant)
        embodied_emissions["vehicle_assembly"]+=get_CO2e(vehicle, emittant)
        embodied_emissions["total_embodied"]+=get_CO2e(total, emittant)
        embodied_emissions["total_embodied"]+=get_CO2e(batt_assembly, emittant)
        embodied_emissions["total_embodied"]+=get_CO2e(batt_recycling, emittant)
    ## for ICE vehicles, get_fuel_carbon calculates CO2e, including upstream CO2e
    ## for grid mix, select CO2e option
    batt_assembly, battery, vehicle, total_embodied = list(embodied_emissions.values())
    usage = get_usage_emissions(Vehicle, vehicle_fuel_economy, vehicle_class, miles_estimated, grid_intensity)
    lifetime_emissions.append([batt_assembly, battery, vehicle, total_embodied, usage, (total_embodied + usage)])
    return lifetime_emissions

    

## Calculating Emissions using Average US Grid Mix

In [8]:
## Approach: since here we aren't taking a specific zip code, average C02 emissions across all zip codes
## then for a given vehicle type calculate in-use emissions according to efficiency*carbon_intensity
## for ICE vehicles, use mpge*cabon_intensity_petrol
## for EVs, use average grid mix intensity
## for PHEVs, use mpge*carbon_intensity_petrol, assume combined cycle driving

In [9]:
vehicle_data.columns

Index(['level_0', 'Model', 'Make', 'Year', 'AdjWeight', 'match_index',
       'baseModel', 'displ', 'cylinders', 'trany', 'comb08', 'combinedUF',
       'combE', 'highway08', 'city08', 'UCity', 'UHighway', 'atvType',
       'rangeHwyA', 'RangeHwy', 'Range', 'MFRCode', 'VehicleID'],
      dtype='object')

In [14]:
## CO2e calculation
## for CO2e we use CO2, CH4, N2O
emissions_array = []
for i in range(len(vehicle_data)):
    Vehicle = vehicle_data.iloc[i]
    lifetime_emissions = process_vehicle(Vehicle, vehicle_emissions_data, LI_emissions_data, NIMH_emissions_data, ['CO2', 'CH4', 'N2O'], 178000, avg_CO2eRate)
    emissions_array.append(lifetime_emissions)
emissions_array = np.array(emissions_array).squeeze()

In [15]:
## CO2e mean
np.mean(emissions_array, axis = 0)

array([2.05978319e+04, 1.50269231e+05, 6.60226825e+06, 8.28604617e+06,
       9.21082725e+07, 1.00394319e+08])

In [16]:
## mean before
## (usage emissions doesn't change since grid mix assumes CO2e already and gas emissions assume CO2e~CO2)
np.mean(emissions_array, axis = 0)

array([2.05978319e+04, 1.50269231e+05, 6.60226825e+06, 8.28604617e+06,
       9.21082725e+07, 1.00394319e+08])

### Add emissions estimates to vehicle dataframe

In [17]:
emissionsdf = pd.DataFrame(emissions_array)

In [18]:
emissionsdf.columns = ['battery_assembly', 'battery_manufacture', 
                       'vehicle_manufacture', 'total_embodied', 'usage', 'total']

In [19]:
emissionsjoin = pd.merge(vehicle_data, emissionsdf, left_index = True, right_index = True)

In [20]:
emissionsjoin

,level_0,Model,Make,Year,AdjWeight,match_index,baseModel,displ,cylinders,trany,...,RangeHwy,Range,MFRCode,VehicleID,battery_assembly,battery_manufacture,vehicle_manufacture,total_embodied,usage,total
0,0.0,NSX,Acura,2000,3500.0,1872.0,nsx,3.0,6.0,Automatic 4-spd,...,0.0,0,NaN,0,0.0,60000.000000,5.506655e+06,7.074855e+06,1.066433e+08,1.137182e+08
1,1.0,NSX,Acura,2000,3500.0,1872.0,nsx,3.2,6.0,Manual 6-spd,...,0.0,0,NaN,1,0.0,60000.000000,5.506655e+06,7.074855e+06,1.066433e+08,1.137182e+08
2,2.0,M Coupe,BMW,2000,3250.0,1736.0,m,3.2,6.0,Manual 5-spd,...,0.0,0,NaN,2,0.0,60000.000000,5.113323e+06,6.681523e+06,9.902593e+07,1.057075e+08
3,3.0,TT Coupe,Audi,2000,3250.0,1736.0,tt coupe,1.8,4.0,Manual 5-spd,...,0.0,0,NaN,3,0.0,60000.000000,5.113323e+06,6.681523e+06,8.205006e+07,8.873158e+07
4,4.0,Z3 Coupe,BMW,2000,3312.5,1751.0,z3,2.8,6.0,Automatic 4-spd,...,0.0,0,NaN,4,0.0,60000.000000,5.211656e+06,6.779856e+06,9.641374e+07,1.031936e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25148,NaN,Nexo Blue,Hyundai,2019,4000.0,NaN,NaN,0.0,0.0,Automatic (A1),...,380.0,0,NaN,25202,0.0,437873.289281,7.403872e+06,8.882745e+06,3.755846e+07,4.644121e+07
25149,NaN,Nexo Blue,Hyundai,2020,4000.0,NaN,NaN,0.0,0.0,Automatic (A1),...,380.0,0,NaN,25203,0.0,437873.289281,7.403872e+06,8.882745e+06,3.755846e+07,4.644121e+07
25150,NaN,Nexo Blue,Hyundai,2021,4000.0,NaN,NaN,0.0,0.0,Automatic (A1),...,380.0,0,NaN,25204,0.0,437873.289281,7.403872e+06,8.882745e+06,3.755846e+07,4.644121e+07
25151,NaN,Nexo Blue,Hyundai,2022,4000.0,NaN,NaN,0.0,0.0,Automatic (A1),...,380.0,0,NaN,25205,0.0,437873.289281,7.403872e+06,8.882745e+06,3.755846e+07,4.644121e+07


In [330]:
#emissionsjoin.to_csv("emissions_vehicle_join_adjweights_2000-2023.csv", index = False)